In [5]:
import sys
sys.path.append('..')
import conet 

In [6]:
data_generator = dg.DataGenerator()
ratios_distribution = rd.RatiosDistribution([0.2, 0.5, 0.3], [1.0, 3, 2.5], [0.1, 0.6, 0.3], 0.3)
tree = data_generator.generate_random_tree(40, 20)
input_data = data_generator.generate_random_ratios(40, tree, ratios_distribution, 80)
input_data.save("./")

NameError: name 'dg' is not defined